# Predicting Company Bankruptcy using Machine Learning
By CS3244 Project Team 11  
Cao Han  |  Chin Sek Yi  |  Lim Kai Sin  |  Luo Xinming  

## Section 1: Project Overview

### 1.1 Project Motivation / Problem Statement
In today's dynamic business landscape, exemplified by recent events such as the bankruptcy of Silicon Valley Bank, the ability to anticipate and mitigate financial risks is crucial for sustainable growth and stability. This project aims to develop a robust predictive model of company bankruptcy, leveraging advanced machine learning algorithms and financial data analysis techniques, so as to equip stakeholders with nuanced insights to confidently traverse the unpredictable landscape of financial risk.

### 1.2 Dataset Description
The dataset used in this project is about bankruptcy prediction of Polish companies. The dataset contains financial rates from one year and corresponding class label that indicates bankruptcy status 3 years after that year. The data contains 10503 instances (financial statements), with 495 representing bankrupted companies and 10008 still operating at the end of the 3-year forecasting period.

The data was collected from [Emerging Markets Information Service](http://www.securities.com), which is a database containing information on emerging markets around the world. The bankrupt companies were analyzed in the period 2000-2012, while the still operating companies were evaluated from 2007 to 2013.  

Source: [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/dataset/365/polish+companies+bankruptcy+data)

### 1.3 Methodologies
The proposed solution involves employing a diverse array of predictive models, including logistic regression, k nearest neighbors and decision trees, each tailored to identify companies at risk of bankruptcy. To gauge effectiveness, logistic regression will serve as the benchmark classifier, ensuring a thorough evaluation of alternative models. We will also leverage ensemble methods including bagging, boosting and random forests to amplify our model's predictive capabilities, while enriching our understanding of machine learning techniques.

## Section 2: Dataset collection 
The original dataset is in the ARFF format. We use panda library to convert and store the dataset

# Step 2: Data Exploration (EDA)

# Step 3: Data pre-processing 

# Step 4: Data Modelling

# Step 6: Numerical results